In [1219]:
colors = ColorData[97, "ColorList"];

In [1]:
d = Import["../data/purchase_times.csv"];
d = Sort[FromDateString /@ Flatten[d]];

In [3]:
d1 = FromDateString["2024-11-01 00:00:00.000"];
d2 = FromDateString["2024-11-02 00:00:00.000"];
d3 = FromDateString["2024-11-03 00:00:00.000"];
d4 = FromDateString["2024-11-04 00:00:00.000"];
d5 = FromDateString["2024-11-05 00:00:00.000"];
d6 = FromDateString["2024-11-06 00:00:00.000"];
d7 = FromDateString["2024-11-07 00:00:00.000"];
d8 = FromDateString["2024-11-08 00:00:00.000"];
d31 = FromDateString["2024-12-01 00:00:00.000"];

In [12]:
DateToReal[start_DateObject, date_DateObject] := N[2*Pi*QuantityMagnitude[DateDifference[d1,date], "Days"]]

In [13]:
dReal = Table[DateToReal[d1, i] ,{i, d}];
T = DateToReal[d1, d31];

In [15]:
jsonRep = {"N" -> Length[dReal], "T" -> T, "t" -> dReal};

In [1600]:
Export["../data/purchase_times.json", jsonRep];

In [17]:
dfiltered1 = Select[d, d1 <= # < d2 &];
dfiltered7 = Select[d, d1 <= # < d8 &];

In [924]:
es = EventSeries[ {#, 1} & /@ d];
es1 = EventSeries[ {#, 1} & /@ dfiltered1];
es7 = EventSeries[ {#, 1} & /@ dfiltered7];

In [1595]:
TimelinePlot[es1, PlotStyle->Tiny, PlotRange->{d1,d2}];

In [1590]:
ListPlot[es7, 
  Filling->Axis, PlotRange -> {{d1, d8}, {0,1}},
  Ticks->{Automatic, None},
  PlotMarkers->" ", 
  Axes->{Automatic,None},
  Ticks->{{d1,d2,d3,d4,d5,d6,d7,d8}, Automatic}
  ];

In [1585]:
ListPlot[es, Filling->Axis, PlotRange -> {{d1, d31}, {0,1}},
  Ticks->{Automatic, None}, PlotMarkers->" ", Axes->{Automatic,None}];

In [1575]:
rateFun = alpha*Exp[Cos[t-phi]-1]^sigma;

In [32]:
rateIntegral = Integrate[rateFun, {t, 0, 60*Pi}]

60 alpha Pi BesselI[0, sigma]
-----------------------------
            sigma
           E

In [50]:
stepData = es7["Dates"];
end = d8;
stepFunction  = TimeSeries[
    Append[Transpose[{Prepend[stepData, d1], Range[0, Length[stepData]]}], {end, Length[stepData]}]
];


In [1570]:
stepPlot = DateListPlot[
    stepFunction,
    InterpolationOrder -> 0,
    PlotRange -> {{d1,d8}, Automatic},
    GridLines -> {True,False},
    PlotStyle -> Directive[AbsoluteThickness[1]],
    Frame -> True,
    Background -> None,
    FrameStyle -> Large,
    FrameTicks -> {{Automatic, None},  {{{d1, "Day 1"}, {d2, "Day 2"}, {d3, "Day 3"}, {d4, "Day 4"}, {d5, "Day 5"}, {d6, "Day 6"}, {d7, "Day 7"}, {d8, "Day 8"}}, None}},
    ImageSize -> 1024
];

In [1565]:
Export["../images/step_plot_sales.png", stepPlot];

In [1057]:
{bins, countsD} = HistogramList[dReal, {0, 14*Pi, 2*Pi/24}];

In [1560]:
salesHourlyHistogram = DateHistogram[
    es7, "Hour", 
    PlotRange -> {{d1, d8}, Automatic},
    Frame -> True,
    Background -> None,
    ChartStyle -> colors[[1]],
    FrameTicks -> {{Automatic, None},  {{{d1, "Day 1"}, {d2, "Day 2"}, {d3, "Day 3"}, {d4, "Day 4"}, {d5, "Day 5"}, {d6, "Day 6"}, {d7, "Day 7"}, {d8, "Day 8"}}, None}},
    FrameStyle-> Large,
    ImageSize -> 1024
];

In [1555]:
Export["../images/sales_hourly_histogram.png", salesHourlyHistogram];

In [498]:
fitdata = Select[dReal, # < 14*Pi&]; 
n=Length[fitdata];
loglike = sigma*Total[Cos[fitdata - phi]] + (Log[alpha]-sigma)*n - alpha*14*Pi*Exp[-sigma]*BesselI[0, sigma];

In [633]:
{crit, learnedParams} = NMaximize[{loglike, sigma > 0 && alpha > 0 && -Pi < phi <= Pi}, {alpha,sigma,phi}];

{379.774, {alpha -> 14.7926, sigma -> 0.878421, phi -> -2.1888}}

In [728]:
likehomog = lambda^n*Exp[-lambda*14*Pi];
{crith, paramh} = NMaximize[{likehomog, lambda > 0}, lambda];

45
{9.32649 10  , {lambda -> 1.75667}}

In [1072]:
stepHour = 2*Pi/24;
stepMin = stepHour/60;
processCountsMin = Table[NIntegrate[rateFun /. learnedParams, {t, 0, x}], {x, 0, 14*Pi, stepMin}];
processCountsHour = Table[NIntegrate[rateFun /. learnedParams, {t, 0, x}], {x, 0, 14*Pi, stepHour}];

tsfit = TimeSeries[processCountsMin, {d1, d8}];


In [1540]:
tshomog = TimeSeries[Table[x/lambda*4*Pi /. paramh, {x, 0, 14*Pi, step}], {d1, d8}];

In [1141]:
countFit = Table[
    NIntegrate[rateFun /. learnedParams, {t, i*stepHour, (i+1)*stepHour}],
    {i, 0, 7*24-1}
];

In [1545]:
salesHourlyHistogram2 = Show[
    BarChart[countsD, ChartStyle -> Directive[colors[[1]], Opacity[0.7]],     Frame->True,
      FrameTicks -> {{Automatic, None},  {{{0, "Day 1"}, {24, "Day 2"}, {24*2, "Day 3"}, {24*3, "Day 4"}, {24*4, "Day 5"}, {24*5, "Day 6"}, {24*6, "Day 7"}, {24*7, "Day 8"}}, None}}
    ],
    BarChart[countFit, ChartStyle -> Directive[colors[[2]], Opacity[0.7]]],
    ImageSize -> 1024,
    FrameStyle -> Large
];

In [1455]:
Export["assets/images/sales_hourly_histogram_fit.png", salesHourlyHistogram2];

assets/images/sales_hourly_histogram_fit.png

In [1550]:
stepPlot2 = DateListPlot[
    {tshomog, stepFunction, tsfit},
    InterpolationOrder -> 0,
    PlotRange -> {{d1,d8}, {0,400}},
    GridLines -> {True,False},
    PlotStyle -> {{Directive[AbsoluteThickness[1]], colors[[3]]} , {Directive[AbsoluteThickness[1]], colors[[1]]}, {Directive[AbsoluteThickness[1]], colors[[2]]}},
    Frame -> True,
    Background -> None,
    FrameTicks -> {{Automatic, None},  {{{d1, "Day 1"}, {d2, "Day 2"}, {d3, "Day 3"}, {d4, "Day 4"}, {d5, "Day 5"}, {d6, "Day 6"}, {d7, "Day 7"}, {d8, "Day 8"}}, None}},
    PlotLegends -> Placed[{Style["homogeneous fit", Large], Style["data", Large], Style["fit", Large]}, Below],
    ImageSize -> 1024,
    FrameStyle -> Large
];

In [1535]:
Export["assets/images/step_plot_sales_fit.png", stepPlot2];

assets/images/step_plot_sales_fit.png